In [1]:
import tensorflow as tf
import cifar10_input

import numpy as np
import time

max_steps = 300
batch_size = 128

#定义截断正太分布，初始化权重，并添加Ｌ2正则化项
def variable_with_weight_loss(shape,stddev,wl):
    var = tf.Variable(tf.truncated_normal(shape,stddev=stddev))
    if wl is not None:
        weight_loss = tf.multiply(tf.nn.l2_loss(var),wl,name='weight_loss')
        tf.add_to_collection('losses',weight_loss)
    return var

images_trains, labels_trains = cifar10_input.distorted_inputs(data_dir="./cifar_data/cifar-10-batches-bin/",
                                                              batch_size=batch_size)

images_test, labels_test = cifar10_input.inputs(eval_data=True,data_dir="./cifar_data/cifar-10-batches-bin/",
                                               batch_size=batch_size)

#定义输入数据的　placeholder
image_holder = tf.placeholder(tf.float32,[batch_size,24,24,3])
label_holder = tf.placeholder(tf.int32,[batch_size])

#定义第一个卷积层
weight1 = variable_with_weight_loss(shape=[5,5,3,64],stddev=5e-2,wl=0.0)
kernerl1 =tf.nn.conv2d(image_holder,weight1,[1,1,1,1],padding="SAME")
bias1 = tf.Variable(tf.constant(0.0, shape=[64]))
conv1 = tf.nn.relu(tf.nn.bias_add(kernerl1,bias1))
pool1 = tf.nn.max_pool(conv1,ksize=[1,3,3,1], strides=[1,2,2,1],padding="SAME")
norm1 = tf.nn.lrn(pool1, bias=1.0, alpha=0.001/9.0, beta=0.75)

#第二个卷积层
weight2 = variable_with_weight_loss(shape=[5,5,64,64],stddev=5e-2, wl=0.0)
kernerl2 = tf.nn.conv2d(norm1, weight2, [1,1,1,1], padding="SAME")
bias2 = tf.Variable(tf.constant(0.1,shape=[64]))
conv2 = tf.nn.relu(tf.nn.bias_add(kernerl2,bias2))
norm2 = tf.nn.lrn(conv2,bias=1.0, alpha=0.001/9.0, beta=0.75)
pool2 = tf.nn.max_pool(norm2,ksize=[1,3,3,1],strides=[1,2,2,1], padding="SAME")

#全连接层
reshape = tf.reshape(pool2,[batch_size,-1])
dim = reshape.get_shape()[1].value
weight3 = variable_with_weight_loss(shape=[dim,384],stddev=0.04, wl=0.004)
bias3 = tf.Variable(tf.constant(0.1, shape=[384]))
local3 = tf.nn.relu(tf.matmul(reshape,weight3)+bias3)

#第二个全连接层
weight4 = variable_with_weight_loss(shape=[384,192], stddev=0.04, wl = 0.004)
bias4 = tf.Variable(tf.constant(0.1,shape=[192]))
local4 = tf.nn.relu(tf.matmul(local3,weight4)+bias4)

#最后一层
weight5 = variable_with_weight_loss(shape=[192,10],stddev=1/192.0, wl=0.0)
bias5 = tf.Variable(tf.constant(0.0, shape=[10]))
logits = tf.add(tf.matmul(local4,weight5),bias5)

def loss(logits, labels):
    labels = tf.cast(labels,tf.int64)
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels, 
                                                                   name='cross_entropy_example')
    cross_entropy_mean = tf.reduce_mean(cross_entropy,name='cross_entropy')
    tf.add_to_collection('losses',cross_entropy_mean)
    
    return tf.add_n(tf.get_collection('losses'),name='total_loss')

loss = loss(logits, label_holder)

train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

top_k_op = tf.nn.in_top_k(logits, label_holder,1)

sess = tf.InteractiveSession()

tf.global_variables_initializer().run()

tf.train.start_queue_runners()

for step in range(1000):
    start_time = time.time()
    image_batch, label_batch = sess.run([images_trains, labels_trains])
    _, loss_value = sess.run([train_op, loss],feed_dict={image_holder: image_batch, label_holder: label_batch})
    duration = time.time() - start_time
    if step % 10 ==0:
        examples_per_sec = batch_size/duration
        sec_per_batch = float(duration)
        
        format_str = ('step %d, loss=%.2f (%.1f examples/sec; %.3f sec/batch)')
        print(format_str % (step, loss_value, examples_per_sec, sec_per_batch))

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.FixedLengthRecordDataset`.
Filling queue with 20000 CIF

step 820, loss=1.21 (868.0 examples/sec; 0.147 sec/batch)
step 830, loss=1.47 (1027.4 examples/sec; 0.125 sec/batch)
step 840, loss=1.32 (1058.2 examples/sec; 0.121 sec/batch)
step 850, loss=1.29 (958.2 examples/sec; 0.134 sec/batch)
step 860, loss=1.37 (987.0 examples/sec; 0.130 sec/batch)
step 870, loss=1.55 (1022.3 examples/sec; 0.125 sec/batch)
step 880, loss=1.36 (842.3 examples/sec; 0.152 sec/batch)
step 890, loss=1.37 (1014.3 examples/sec; 0.126 sec/batch)
step 900, loss=1.25 (1006.5 examples/sec; 0.127 sec/batch)
step 910, loss=1.30 (938.9 examples/sec; 0.136 sec/batch)
step 920, loss=1.37 (966.2 examples/sec; 0.132 sec/batch)
step 930, loss=1.15 (1042.6 examples/sec; 0.123 sec/batch)
step 940, loss=1.21 (877.6 examples/sec; 0.146 sec/batch)
step 950, loss=1.23 (698.2 examples/sec; 0.183 sec/batch)
step 960, loss=1.25 (1120.1 examples/sec; 0.114 sec/batch)
step 970, loss=1.13 (951.3 examples/sec; 0.135 sec/batch)
step 980, loss=1.28 (928.5 examples/sec; 0.138 sec/batch)
step 99

In [2]:
num_examples = 10000
import math
num_iter = int(math.ceil(num_examples/batch_size))
true_count = 0
total_sample_count = num_iter * batch_size
step = 0
while step < num_iter:
    image_batch, label_batch = sess.run([images_test, labels_test])
    prediction = sess.run([top_k_op], feed_dict={image_holder:image_batch, label_holder:label_batch})
    
    true_count += np.sum(prediction)
    step +=1

In [19]:
precision = true_count/ total_sample_count
print('precision = %.3f' %precision)

precision = 0.632
